Articles I've been reading for NLP


https://www.geeksforgeeks.org/twitter-sentiment-analysis-using-python/

https://codereview.stackexchange.com/questions/181152/identify-and-extract-urls-from-text-corpus

https://en.wikipedia.org/wiki/Naive_Bayes_classifier

https://streamhacker.com/2010/05/10/text-classification-sentiment-analysis-naive-bayes-classifier/

https://nlpforhackers.io/topic-modeling/

https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21

https://towardsdatascience.com/sentiment-analysis-with-python-part-1-5ce197074184

http://www.nltk.org/book/ch06.html

https://medium.freecodecamp.org/basic-data-analysis-on-twitter-with-python-251c2a85062e 


In [104]:
import pandas as pd
import numpy as np
import langdetect
#either pip install langdetect or conda install -c conda-forge langdetect
from langdetect import detect
from pandas.plotting import scatter_matrix
import re
#conda install -c conda-forge textblob 
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import html
from html.parser import HTMLParser
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import textblob
from textblob import TextBlob
import string

In [10]:
#read in all the tweets from geniuine accounts
tweets = pd.read_csv('~/Downloads/datasets_full/datasets_full.csv/genuine_accounts.csv/tweets.csv')


#schema from tweets.info()
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2839362 entries, 0 to 2839361
Data columns (total 25 columns):
id                         object
text                       object
source                     object
user_id                    float64
truncated                  float64
in_reply_to_status_id      float64
in_reply_to_user_id        float64
in_reply_to_screen_name    object
retweeted_status_id        float64
geo                        float64
place                      object
contributors               float64
retweet_count              float64
reply_count                float64
favorite_count             float64
favorited                  float64
retweeted                  float64
possibly_sensitive         float64
num_hashtags               float64
num_urls                   float64
num_mentions               float64
created_at                 object
timestamp                  object
crawled_at                 object
updated                    object
dtypes: float64(16), object(9)
memory usage: 541.6+ MB

In [88]:
def df_detect_en(df, text_col):
    
    '''Input is a dataframe (df) and name of the column (text_col) to check for english 
    This function creates a new Boolean column called "en_flag"
    "en_flag" is True if the text_col column is detected as "en" 
    Dataframe with the new column is returned.
    '''
    
    def detect_en(x):
        #assumes you have langdetect imported
        flag = False
        lang = detect(text)
        if lang=='en':
            flag = True
        return flag
    
    df[text_col] = df[text_col].astype(str)
    df['en_flag'] = df.loc[text_col].apply(lambda x: detect_en(x))
    return(df)

In [66]:
#Functions
class MLStripper(HTMLParser):
    #https://docs.python.org/3/library/html.parser.html
    #https://stackoverflow.com/questions/11061058/using-htmlparser-in-python-3-2
    def __init__(self):
        super().__init__()
        self.reset()
        self.fed = []
       
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def remove_swords(text):
    #https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
    stop_words = set(stopwords.words('english'))   
    word_tokens = word_tokenize(text)  
    filtered_sentence = [w for w in word_tokens if not w in stop_words]   
    return filtered_sentence

def get_tweet_sentiment(tweet): 
        ''' 
        https://medium.freecodecamp.org/basic-data-analysis-on-twitter-with-python-251c2a85062e 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(tweet) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'

def lemmatize(text):
    #https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21
    text_out = []
    def get_lemma(word):
        lemma = wn.morphy(word)
        if lemma is None:
            return word
        else:
            return lemma
    
    for word in text:
        lword = get_lemma(word)
        text_out.append(lword)
    return text_out
    

blah_ids['text'] = blah_ids['text'].astype(str)

In [143]:
def nlp_clean(text):
    #get rid of #retweets RT @[\S]+  mentions @[\S]+ urls http:\S+|https\S+|www.\S+ punctuation
    text_out = []
    result = ''
    try:
        result = re.sub(r"RT @[\S]+: |@[\S]+|http:\S+|https\S+|www.\S+|[^\w\s]", "", text) 
        
         #to lower case
        result = result.lower()
        
        #get rid of url encoding
        #https://stackoverflow.com/questions/11061058/using-htmlparser-in-python-3-2
        result = strip_tags(result)

        #get rid of special ascii characters
        result = ''.join([c for c in result if ord(c) < 128])

       
        #get rid of stopwords
        result = remove_swords(result)

        #get word roots
        result = lemmatize(result)
        
    except:
        text_out = ['Failed']
    
    return result

#get sentiment
def sentiment(text):
    sentiment = get_tweet_sentiment(text)
    return sentiment 

In [144]:
def clean_tweets(df, text_col):
    #creates two new features
    #word_bag is our bag of words that has been cleaned
    #sentiment is the sentiment for the individual tweet
    df['word_bag'] = df[text_col].apply(lambda x: nlp_clean(x))
    df['sentiment'] = df[text_col].apply(lambda x: sentiment(x))
    return(df) 

In [145]:
#This filters for all the tweets from two users (6396 total tweets)  
two_ids = tweets.loc[(tweets['user_id'] == 678033) | (tweets['user_id'] == 722623)]
len(two_ids)

6396

In [146]:
test_clean = clean_tweets(two_ids,'text')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [147]:
test_clean['word_bag'][100]

['still',
 'part',
 'group',
 'call',
 'end',
 'humanize',
 'capital',
 'lay',
 'waste',
 'impulse',
 'leading']

In [167]:
#df.groupby('reciept')['prod_name'].count()
df = test_clean.groupby(['user_id','sentiment']).count()
df.index

MultiIndex(levels=[[678033.0, 722623.0], ['negative', 'neutral', 'positive']],
           labels=[[0, 0, 0, 1, 1, 1], [0, 1, 2, 0, 1, 2]],
           names=['user_id', 'sentiment'])